In [ ]:
import optuna
import numpy as np
import tensorflow as tf
from tensorflow import keras

### Constants and hyperparameters

In [ ]:
epochs = 350 # Number of training epochs
split_percentage = 0.8 # Training and test set splitting percentage
validation_split = 0.2 # Validation set percentage
early_stopping_patience = 20 # Number of epochs of patience before triggering early stopping
naca_numbers = ['maximum_camber', 'maximum_camber_position', 'maximum_thickness'] # NACA numbers to predict

In [ ]:
dataset_path = "../Dataset/Flow signals/2c/128.npz" # Dataset path
flow_quantity = "p" # Flow quantity to be used as feature

### Data loading

In [ ]:
# Loading the data
dataset = np.load(dataset_path)
dataset = list(zip(dataset[flow_quantity], dataset["naca_numbers"]))

### Shuffling the dataset

In [ ]:
# Shuffling the dataset
np.random.shuffle(dataset)

### Features and labels

In [ ]:
# Extracting the features and the labels from the dataset
X, Y = zip(*dataset)
X, Y = np.array(X), np.array(Y)

### Training and test set

In [ ]:
# Computing the number of training samples according to the splitting percentage
num_training_samples = int(np.floor(split_percentage * len(dataset)))

In [ ]:
# Extracting the training features and labels
X_train = np.expand_dims(X[:num_training_samples], axis=2)
Y_train = Y[:num_training_samples]

# Extracting the test features and labels
X_test = np.expand_dims(X[num_training_samples:], axis=2)
Y_test = Y[num_training_samples:]

### Data normalization

In [ ]:
# Computing the mean and standard deviation of the training features
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

In [ ]:
# Function to normalize samples
def normalize(x):
    x = (x - mean) / std
    return x

### Creating the study cases

In [ ]:
# Function to create a testing model
def createModel(trial):
    # Creating the Model
    model = keras.Sequential()

    # Dropout rate
    dropout_rate = trial.suggest_discrete_uniform("dropout_rate", 0.01, 0.1, 0.01)

    # Kernels size
    kernel_size = trial.suggest_int("kernel_size", 1, 6)

    # Pools size
    pool_size = trial.suggest_int("pool_size", 2, 4)

    # Input layer
    model.add(keras.layers.InputLayer(input_shape=np.shape(X_train)[1:]))


    # Normalization layer
    model.add(keras.layers.Lambda(normalize))

    # Number of convolutional layers
    num_conv_layers = trial.suggest_int("num_conv_layers", 1, 2)
    for i in range(num_conv_layers):
        # Number of filters
        num_filters = trial.suggest_categorical(f"num_filters__conv_layer_{i+1}", [2*j for j in range(1, 10)])

        # Adding the convolutional layer
        model.add(keras.layers.Conv1D(filters=num_filters, kernel_size=kernel_size, activation=tf.nn.tanh, padding="same"))
        model.add(keras.layers.MaxPool1D(pool_size=pool_size, padding="same"))
        model.add(keras.layers.Dropout(rate=dropout_rate))

    # Flatten layer
    model.add(keras.layers.Flatten())

    # Number of fully connected layers
    num_fc_layers = trial.suggest_int("num_fc_layers", 1, 5)
    for i in range(num_fc_layers):
        # Number of units
        num_units = trial.suggest_categorical(f"num_units__fc_layer_{i+1}", [10*j for j in range(1, 4)])

        # Adding the fully connected layer
        model.add(keras.layers.Dense(num_units, activation=tf.nn.tanh))

    # Output layer
    model.add(keras.layers.Dense(len(naca_numbers)))

    # Compiling the model
    model.compile(loss='mse', optimizer="adam", metrics=['mae'])

    return model

In [ ]:
# Early stopping with a predefined patience
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=early_stopping_patience,
    restore_best_weights=True,
    verbose=False
)

In [ ]:
# Function to train the model
def train(model, trial):
    # Batch size 
    batch_size = trial.suggest_categorical("batch_size", [4*i for i in range (1, 6)])

    # Fitting the model
    model.fit(
        X_train, 
        Y_train,
        epochs=epochs,
        validation_split=0.2,
        batch_size=batch_size,
        verbose=0,
        callbacks=[
            early_stopping, 
            optuna.integration.TFKerasPruningCallback(trial, 'val_mae')
        ]
    )

In [ ]:
# Function to evaluate the model
def evaluate(model):
    loss, mae = model.evaluate(X_test, Y_test, verbose=0)
    return loss, mae

In [ ]:
# Objective function to be minimized
def objective(trial):
    # Building the model
    model = createModel(trial)

    # Training the model
    train(model, trial)

    # Evaluating the model
    _, mae = evaluate(model)

    return mae

### Evaluating the hyperparameters

In [ ]:
# Creating the study object with the specified configurations
study = optuna.create_study(
    study_name="CNN",
    direction="minimize",
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=20),
    sampler=optuna.samplers.TPESampler()
)

# Running the study
study.optimize(objective, n_trials=100)

In [ ]:
# Extractig the pruned and complete trials
pruned_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.COMPLETE]

print("STUDY STATISTICS")
print(f"Number of finished trials --> {len(study.trials)}")
print(f"Number of pruned trials --> {len(pruned_trials)}")
print(f"Number of complete trials --> {len(complete_trials)}")

In [ ]:
# Extracting the best trial from the study performed
trial = study.best_trial

# Displaying the obtained results
print("BEST TRIAL")
print(f"Mean Absolute Error --> {trial.value}\n")

print("BEST HYPERPARAMETERS")
for key, value in trial.params.items():
    print(f"{key}: {value}")